In [1]:
from __future__ import division
import numpy as np, numpy.random as nr, numpy.linalg as nlg
import scipy as sp, scipy.linalg as slg, scipy.io as sio, scipy.sparse as ss
# import matplotlib.pyplot as plt

from multiprocessing import Pool

import time
import os, os.path as osp
import csv
import cPickle as pick
# import sqlparse as sql

import activeSearchInterface as ASI
import competitorsInterface as CI

import data_utils as du
import graph_utils as gu
import gaussianRandomFeatures as GRF

import lapsvmp as LapSVM
import anchorGraph as AG

import IPython


In [105]:
t1 = time.time()
X0,Y0,classes = du.load_covertype(sparse=True, normalize=False)
print(time.time() - t1)

NameError: name 'bias_normalize_ft' is not defined

In [111]:
X1 = du.matrix_squeeze(du.bias_normalize_ft (X0, sparse=True).todense())


In [145]:
Xd = du.matrix_squeeze(X0.todense())

In [146]:
Xw, W = du.whiten_data(Xd, sparse=False, rtn_W=True, thresh=1e-6)

In [147]:
np.min(Xw.dot(Xw.T)-np.eye(52)*581012.)

-9.2478119768202305e-05

In [186]:
gammak1=1
# rfc1 = GRF.GaussianRandomFeatures(X1.shape[0], X1.shape[0]*10, gammak=gammak1)
gammak2=10
rfc2 = GRF.GaussianRandomFeatures(Xw.shape[0], Xw.shape[0]*10, gammak=gammak2)

In [174]:
# X2, Y2 = du.stratified_sample(X0, Y0, classes=[0,1], strat_frac=0.1, return_inds=False)
X2, Y2 = X1, Y0

In [187]:
t1 = time.time()
# RX2 = rfc1.computeRandomFeatures(X2.T).T
RX3 = rfc2.computeRandomFeatures(Xw.T).T
print(time.time() - t1)

17.9132378101


In [188]:
i1, i2 = 2012, 1988
print np.exp(-gammak2*nlg.norm(Xw[:,i1]-Xw[:,i2])**2)
print RX3[:,i1].T.dot(RX3[:,i2])

1.63427637131e-32
-0.000275878355976


In [196]:
rfc.SaveToFile('./grf_covtype.cpk')

In [190]:
pi = Y2.sum() / Y2.shape[0]
sparse1 = True
sparse2 = False
verbose = True
alpha = 0.0

prms1 = ASI.Parameters(pi=pi, sparse=sparse1, verbose=verbose)
prms2 = ASI.Parameters(pi=pi, sparse=sparse2, verbose=verbose, alpha=alpha)

wnprms1 = ASI.WNParameters(normalize=True, sparse=sparse1, verbose=verbose)
wnprms2 = ASI.WNParameters(normalize=True, sparse=sparse2, verbose=verbose)

In [202]:
kAS1 = ASI.linearizedAS(prms2)
kAS2 = ASI.linearizedAS(prms2)
wnAS1 = ASI.weightedNeighborAS(wnprms2)
wnAS2 = ASI.weightedNeighborAS(wnprms2)

In [203]:
np_init = 1
nn_init = 0
Ynz = Y2.nonzero()[0]
Yz = (Y2==0).nonzero()[0]
initp_pt = Ynz[nr.choice(len(Ynz), np_init, replace=False)]
initn_pt = Yz[nr.choice(len(Yz), nn_init, replace=False)]
n_init = np_init + nn_init

init_labels = {p:1 for p in initp_pt}
init_labels.update({p:0 for p in initn_pt})
# init_labels.update({390748:0})

In [204]:
kAS1.initialize(RX2, init_labels)
# kAS2.initialize(RX3, init_labels)
wnAS1.initialize(RX2, init_labels)
# wnAS2.initialize(RX3, init_labels)

Constructing C
Time for constructing C: 3.78883099556
Inverting C
Time for inverse: 0.554513931274
Done with the initialization.
Done with the initialization.


In [205]:
t1 = time.time()
next_inds1 = [kAS1.unlabeled_idxs[idx] for idx in (-kAS1.f[kAS1.unlabeled_idxs]).argsort()[:100]]
# next_inds2 = [kAS2.unlabeled_idxs[idx] for idx in (-kAS2.f[kAS2.unlabeled_idxs]).argsort()[:100]]
next_inds2 = [wnAS1.unlabeled_idxs[idx] for idx in (-wnAS1.f).argsort()[:100]]
print(Y2[next_inds1].sum(), Y2[next_inds2].sum())
print(time.time() - t1)

(2, 0)
0.130230903625


In [206]:
hits1 = [n_init]
hits2 = [n_init]
hits3 = [n_init]
hits4 = [n_init]
K = 100
for _ in range(K):
    idx1 = kAS1.getNextMessage()
    kAS1.setLabelCurrent(Y2[idx1])
    hits1.append(hits1[-1]+Y2[idx1])
    
#     idx2 = kAS2.getNextMessage()
#     kAS2.setLabelCurrent(Y2[idx2])
#     hits2.append(hits2[-1]+Y2[idx2])

    idx3 = wnAS1.getNextMessage()
    wnAS1.setLabelCurrent(Y2[idx3])
    hits3.append(hits3[-1]+Y2[idx3])
    
#     idx4 = wnAS2.getNextMessage()
#     wnAS2.setLabelCurrent(Y2[idx4])
#     hits4.append(hits4[-1]+Y2[idx4])
    print('')

Iter: 1, Selected: 259847, Hits: 1, Time: 0.401110
NNAS -- Iter: 1, Selected: 0, Hits: 1, Time: 4.246028

Iter: 2, Selected: 3893, Hits: 2, Time: 0.347102
NNAS -- Iter: 2, Selected: 208517, Hits: 1, Time: 4.097563

Iter: 3, Selected: 247710, Hits: 2, Time: 0.332449
NNAS -- Iter: 3, Selected: 280005, Hits: 1, Time: 3.882671

Iter: 4, Selected: 10807, Hits: 2, Time: 0.345297
NNAS -- Iter: 4, Selected: 362662, Hits: 1, Time: 4.739290

Iter: 5, Selected: 260558, Hits: 2, Time: 0.383065
NNAS -- Iter: 5, Selected: 270695, Hits: 2, Time: 4.108474

Iter: 6, Selected: 12512, Hits: 3, Time: 0.341536
NNAS -- Iter: 6, Selected: 269174, Hits: 3, Time: 3.767402

Iter: 7, Selected: 268640, Hits: 4, Time: 0.348572
NNAS -- Iter: 7, Selected: 12613, Hits: 4, Time: 3.811460

Iter: 8, Selected: 51466, Hits: 4, Time: 0.340144
NNAS -- Iter: 8, Selected: 4019, Hits: 5, Time: 3.872273

Iter: 9, Selected: 3500, Hits: 5, Time: 0.324390
NNAS -- Iter: 9, Selected: 3575, Hits: 6, Time: 4.243873

Iter: 10, Selected

KeyboardInterrupt: 

In [98]:
next_inds1 = [kAS2.unlabeled_idxs[idx] for idx in (-kAS2.f[kAS2.unlabeled_idxs]).argsort()[:100]]
next_inds2 = [wnAS2.unlabeled_idxs[idx] for idx in (-wnAS2.f).argsort()[:100]]
print(Y2[next_inds1].sum(), Y2[next_inds2].sum())

(0, 58)


In [392]:
GRF = reload(GRF)

In [415]:
du = reload(du)

In [97]:
pi

0.0047279574260084131

In [515]:
rfc2 = GRF.GaussianRandomFeatures(fl='./grf.cpk')

In [516]:
t1 = time.time()
RX4 = rfc2.computeRandomFeatures(X2.T).T
print(time.time() - t1)

16.1215000153


In [517]:
np.allclose(RX3, RX4)

True